# Building conversational applications with CrewAI

In this notebook, you learn how to use CrewAI to create collaborative AI agents that can work together to solve complex tasks.

This demonstrates the same functionality as the Strands Agents example, but using CrewAI's multi-agent approach.

## Environment setup

In this task, you set up your environment and install the required packages for CrewAI.

In [1]:
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

model_id = "amazon.nova-lite-v1:0"

## Define Tools for CrewAI

In this task, we define the tools that our CrewAI agents will use.

In [2]:
@tool
def calculator_tool(oper1: int, oper2: int, operator: str) -> str:
    """
    Perform basic arithmetic operations on two numbers.
    
    Args:
        oper1: First operand
        oper2: Second operand  
        operator: Operation to perform (+, -, *, / or add, subtract, multiply, divide)
    
    Returns:
        Result of the calculation as a string
    """
    def add(a, b):
        return a + b

    def subtract(a, b):
        return a - b

    def multiply(a, b):
        return a * b

    def divide(a, b):
        if b == 0:
            return "Cannot divide by zero"
        else:
            return float(a / b)
    
    if operator == '+' or operator == 'add':
        return str(add(oper1, oper2))
    elif operator == '-' or operator == 'subtract':
        return str(subtract(oper1, oper2))
    elif operator == '*' or operator == 'multiply':
        return str(multiply(float(oper1), float(oper2)))
    elif operator == '/' or operator == 'divide':
        return str(divide(oper1, oper2))
    else:
        return "Invalid operator"

In [ ]:
@tool
def search_tool(query: str) -> str:
    """
    Search with DuckDuckGo for information about a topic.
    
    Args:
        query: The search query
    
    Returns:
        DuckDuckGo search results as a string
    """
    tool = DuckDuckGoSearchRun()
    result = tool.invoke(query)
    return result

## Create CrewAI Agents and Tasks

Now we create specialized agents for different tasks and define how they work together.

In [4]:
# Create the Bedrock LLM
llm = LLM(
    model="bedrock/us.amazon.nova-lite-v1:0"
)

# Create agent
agent = Agent(
    role="Expert in searching and calculations",
    goal="Use available tools to find Find accurate information on any topic using tools",
    backstory="You are a helpful assistant that can perform calculations and search for information on DuckDuckGo. Use the available tools when needed to provide accurate and helpful responses.",
    tools=[search_tool, calculator_tool],
    llm=llm,
    verbose=True
)

## Test the CrewAI System

Now let's test our CrewAI system with the same queries used in the Strands example.

In [5]:
def run_crew_task(user_query: str):
    """Run a CrewAI task based on user query"""
    
    # Create task
    task = Task(
        description=f"Answer this query: {user_query}",
        expected_output="A comprehensive answer to the user's query",
        agent=agent
    )
    
    # Create crew
    crew = Crew(
        agents=[agent],
        tasks=[task],
        verbose=True
    )
    
    # Execute
    result = crew.kickoff()
    return result

In [6]:
# Test with the same complex query from Strands example
query = "What is Amazon SageMaker? What is launch year multiplied by 2"

print(f"Query: {query}\n")
print("CrewAI Response:")
print("=" * 50)

response = run_crew_task(query)
print(response)

Query: What is Amazon SageMaker? What is launch year multiplied by 2

CrewAI Response:


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 226a45e9-ceea-4b0b-9f09-fc7308c76799                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert in searching and calculations                                                                    │
│                                                                                                                 │
│  Task: Answer this query: What is Amazon SageMaker? What is launch year multiplied by 2                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

The result of the DuckDuckGo search is With Amazon SageMaker Canvas, users can leverage the breadth of AWS services
for data, machine learning, artificial intelligence, generative AI, and business intelligence through a single 
interface. SageMaker Canvas abstracts away complexities, making these powerful technologies... Module 1, “ What is 
Amazon SageMaker ?,” provides a history of the evolution of AI and ML. The benefits of Amazon SageMaker will be 
reviewed and sample use cases are provided. After you’re comfortable with the basics, you'll learn about how Amazon
SageMaker works. Compare Amazon SageMaker and Vertex AI head-to-head across pricing, user satisfaction, and 
features, using data from actual users. The Amazon SageMaker API • Python SDK orchestrating all Amazon SageMaker 
activity • High-level objects for algorithm selection, training, deploying, automatic model tuning, etc. • Amazon 
SageMaker is a cloud-based machine learning (ML) platform that simplifies the process of building, training, and 
deploying machine learning models. Here.

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Expert in searching and calculations                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Amazon SageMaker is a fully managed machine learning service that provides every developer and data scientist  │
│  with the ability to build, train, and deploy machine learning models quickly. Amazon SageMaker removes the     │
│  heavy lifting from each step of the machine learning process to make it easier to develop high-quality         │
│  models. The launch year of Amazon SageMaker is 2017. The launch year multiplied by 2 is 4034.0.                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7b85c1e8-278c-42db-a8e0-b7d39de12906                                                                     │
│  Agent: Expert in searching and calculations                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 226a45e9-ceea-4b0b-9f09-fc7308c76799                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Amazon SageMaker is a fully managed machine learning service that provides every developer and   │
│  data scientist with the ability to build, train, and deploy machine learning models quickly. Amazon SageMaker  │
│  removes the heavy lifting from each step of the machine learning process to make it easier to develop          │
│  high-quality models. The launch year of Amazon SageMaker is 2017. The launch year multiplied by 2 is 4034.0.   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Amazon SageMaker is a fully managed machine learning service that provides every developer and data scientist with the ability to build, train, and deploy machine learning models quickly. Amazon SageMaker removes the heavy lifting from each step of the machine learning process to make it easier to develop high-quality models. The launch year of Amazon SageMaker is 2017. The launch year multiplied by 2 is 4034.0.


In [ ]:
# Test with calculation-only query
calc_query = "What is 156 multiplied by 23?"
print(f"Query: {calc_query}")
print(f"Response: {run_crew_task(calc_query)}")
print("\n" + "="*50 + "\n")

In [ ]:
# Test with search-only query
search_query = "Tell me about artificial intelligence"
print(f"Query: {search_query}")
print(f"Response: {run_crew_task(search_query)}")
print("\n" + "="*50 + "\n")